In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2 #Generates data for binary classification used in Hastie et al. 2009, Example 10.2.
import matplotlib.pyplot as plt

In [2]:
from sklearn.naive_bayes import GaussianNB

In [34]:
df = make_hastie_10_2(n_samples=300)
X=df[0]
y=df[1]


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=13)

In [36]:
len(df[0])

300

In [37]:
y_train

array([ 1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,
        1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,
       -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
        1., -1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,
       -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,
       -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,
        1., -1., -1., -1.,  1.,  1.,  1.])

In [38]:
""" HELPER FUNCTION: GET ERROR RATE ========================================="""
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))
# calculate the average number of errors

In [39]:
""" HELPER FUNCTION: PRINT ERROR RATE ======================================="""
def print_error_rate(err):
    print ('Error rate: Training: %.4f - Test: %.4f' % err)

# print the above

In [40]:
""" HELPER FUNCTION: GENERIC CLASSIFIER ====================================="""
def generic_clf(Y_train, X_train, Y_test, X_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)
#fits train data into given model clf 
#,predicts the test data and returns the error function above for both train and test data

In [41]:
""" ADABOOST IMPLEMENTATION ================================================="""
def adaboost_clf(Y_train, X_train, Y_test, X_test, M, clf):
    n_train, n_test = len(X_train), len(X_test)
    # Initialize weights
    w = np.ones(n_train) / n_train #array of length n_train with 1/n_train in each cell
    pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]#initial. pred arrays
    
    for i in range(M):
        # Fit a classifier with the specific weights
        clf.fit(X_train, Y_train, sample_weight = w) #model clf (variable) with weights w
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)   #predicting new predictions based on the weight
        # Indicator function
        miss = [int(x) for x in (pred_train_i != Y_train)]  #fetching the number of predictions that were correct
        # Equivalent with 1/-1 to update weights
        miss2 = [x if x==1 else -1 for x in miss]  # normalizing said number
        # Error
        err_m = np.dot(w,miss) / sum(w) # calc. weight error as the scalar product of w and miss divided by the sum of weights
        # Alpha
        alpha_m = 0.5 * np.log( (1 - err_m) / float(err_m)) #weight for the classifier 
        # New weights
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        # Add to prediction
        pred_train = [sum(x) for x in zip(pred_train, 
                                          [x * alpha_m for x in pred_train_i])]
        pred_test = [sum(x) for x in zip(pred_test, 
                                         [x * alpha_m for x in pred_test_i])]  # new predictions by adding the weighted prediction
    
    pred_train, pred_test = np.sign(pred_train), np.sign(pred_test)
    # Return error rate in train and test set
    return get_error_rate(pred_train, Y_train), \
           get_error_rate(pred_test, Y_test)

In [42]:
""" PLOT FUNCTION ==========================================================="""
def plot_error_rate(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,450,50))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

# plots the train and test errors against each other

In [45]:
adaboost_clf(y_train, X_train, y_test, X_test, 4, GaussianNB())

(0.07333333333333333, 0.11333333333333333)

In [44]:

GaussianNB().fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)